# Visualizing Eurostat data

In [ ]:
import os
import sys

In [ ]:
import numpy as np
import pandas as pd

import geopandas as gpd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

import geoplot as gplt
import geoplot.crs as gcrs

from tqdm.auto import tqdm

from r_wrapper import eurostat

In [ ]:
sns.set_context('talk')

## Retrieving geographical information

The NUTS classification subdivides each member state into regions at three different levels, covering NUTS 1, 2 and 3 from larger to smaller areas (https://ec.europa.eu/eurostat/web/nuts/background).
Eurostat provides this geographical data in various formats (https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units, looks for "NUTS").

The filename of each dataset follows a specific format:  `<theme>_<spatialtype>_<resolution>_<year>_<projection>_<subset>.<extension>` (check https://ec.europa.eu/eurostat/cache/GISCO/distribution/v2/nuts/nuts-2016-files.html for more information).

To make things easier, we use the `eurostat` package to retrieve this data automatically.

In [ ]:
# hide download progress output
null = open(os.devnull, 'wb')
sys.stderr = null

In [ ]:
df_geo = eurostat.get_eurostat_geospatial(
    output_class='sf', resolution=60, nuts_level=2, year=2016
)
df_geo.head()

In [ ]:
sys.stderr = sys.__stderr__

### Example

To get a feeling for the data, we can plot the geographical composition of an arbitrary country.

In [ ]:
ax = gplt.polyplot(df_geo[df_geo['CNTR_CODE'] == 'DE'])
ax.set_aspect(1.4)

## Retrieving statistical data

Eurostat is a great source for data related to Europe.
An overview of the main tables can be found on https://ec.europa.eu/eurostat/web/regions/data/main-tables.

### Select datasets

In [ ]:
eurostat.search_eurostat('age').head()

In [ ]:
dataset_list = [
    'tgs00101',  # life expectancy
    'tgs00026',  # exposable income
    'tgs00036',  # primary income
    'tgs00010',  # unemployment rate
    'tgs00112',  # touristic bed number
]

### Download and aggregate data

To make the subsequent analysis easier, we download all data at once and store it in a single dataframe.
Additionally, we store associated meta data in another dataframe.

In [ ]:
# hide download progress output
null = open(os.devnull, 'wb')
sys.stderr = null

In [ ]:
df_list = []
meta_list = []

for dataset in tqdm(dataset_list):
    # get dataset description
    dataset_name = eurostat.label_eurostat_tables(dataset)[0]
    print(f'Parsing {dataset} ({dataset_name})')

    # retrieve data
    df_data = eurostat.get_eurostat(dataset, time_format='raw')
    df_data = eurostat.label_eurostat(df_data, code=['geo'], fix_duplicated=True)

    # index rows
    common_columns = ['geo_code', 'geo']
    df_data['idx'] = df_data[common_columns].apply(
        lambda row: '_'.join(row.values.astype(str)), axis=1
    )

    # identify meta information
    info_columns = (
        set(df_data.columns)
        - set(common_columns)
        - {'idx', 'time', 'values', 'info', 'info_short'}
    )

    meta_cols = [
        ';'.join(
            f'{k}={row._asdict()[k]}'
            for k in sorted(row._asdict().keys())
            if k != 'Index'
        )
        for row in df_data[info_columns].itertuples()
    ]
    meta_idx, meta_label = pd.factorize(meta_cols)

    df_data['meta'] = [
        f'{dataset}_{time}_{x}' for x, time in zip(meta_idx, df_data['time'])
    ]

    # pivot data
    df_piv = df_data.pivot(index='idx', columns='meta', values='values')

    # store data
    df_list.append(df_piv)

    meta_list.extend(
        [
            {
                'dataset': dataset,
                'name': dataset_name,
                'meta_idx': idx,
                'meta_label': label,
            }
            for idx, label in zip(np.unique(meta_idx), meta_label)
        ]
    )

Each row corresponds to a geographic region, and each column to a certain statistic.

In [ ]:
sys.stderr = sys.__stderr__

In [ ]:
df_all = pd.concat(df_list, axis=1)
df_all.head()

Extra information for each column is stored in an additional dataframe.

In [ ]:
df_meta = pd.DataFrame(meta_list)
df_meta.head()

In [ ]:
def get_meta(col_name, extended=False):
    dataset, time, idx = col_name.split('_')
    res = df_meta[(df_meta['dataset'] == dataset) & (df_meta['meta_idx'] == int(idx))]
    assert res.shape[0] == 1
    res = res.iloc[0]

    if extended:
        return f'{res["name"]}\n({res["meta_label"]})'
    else:
        return res["name"]

### Overview

To get a rough overview of the data, we can plot the clustered correlation matrix.

In [ ]:
df_corr = df_all.corr()

In [ ]:
non_nan_datasets = set(df_corr.dropna(axis=0).index) & set(
    df_corr.dropna(axis=1).columns
)
df_corr = df_corr.loc[non_nan_datasets, non_nan_datasets]

In [ ]:
# assign unique color to each data source
cluster_names = df_all.columns.str.split('_').str[0].unique()
cluser_colors = cm.rainbow(np.linspace(0, 1, len(cluster_names))).tolist()
cluster_map = {n: c for n, c in zip(cluster_names, cluser_colors)}
cluster_colors = pd.DataFrame(
    [(col, cluster_map[col.split('_')[0]]) for col in df_all.columns]
).set_index(0)[1]

In [ ]:
g = sns.clustermap(
    df_corr,
    xticklabels=False,
    yticklabels=False,
    cmap='vlag_r',
    row_colors=cluster_colors,
    col_colors=cluster_colors,
)

## Data preparation

In order to combine both geographical and statistical data, we merge the data...

In [ ]:
df_all['geo_code'] = [idx.split('_')[0] for idx in df_all.index]

In [ ]:
df_merged = df_geo.merge(df_all, left_on='NUTS_ID', right_on='geo_code')
df_merged.head()

...and dissolve it to reach a resolution which leads to a nice visualization.

In [ ]:
%%time
df_diss = df_merged.dissolve(by='CNTR_CODE', aggfunc='mean')

In [ ]:
df_diss.head()

## Visualization

We visualize the data using choropleth maps. By doing so, each geographical region is colored according to a statistic of interest.

In the following, we will use a single entry from each downloaded dataset.

In [ ]:
time = 2016
hue_selection = [
    f'{row.dataset}_{time}_{row.meta_idx}'
    for row in df_meta.groupby('dataset').apply(lambda x: x.sample(n=1)).itertuples()
]
hue_selection

### Single country

We can get a precise overview of individual regions within a single country.

In [ ]:
size = int(np.ceil(np.sqrt(len(hue_selection))))  # size of plot grid

fig, axes = plt.subplots(nrows=size, ncols=size, figsize=(20, 20))
[ax.axis('off') for ax in axes.ravel()]

for hue, ax in zip(hue_selection, axes.ravel()):
    sub = df_merged[(df_merged['CNTR_CODE'] == 'DE')]

    gplt.choropleth(sub, hue=hue, legend=True, ax=ax)
    ax.set_aspect(1.4)
    ax.set_title(get_meta(hue), fontsize=10)

### Europe

To get a more global picture, we can also plot the aggregated data per country.

In [ ]:
for hue in hue_selection:
    ax = gplt.choropleth(
        df_diss,
        hue=hue,
        legend=True,  # k=5,
        figsize=(16, 12),
        extent=(
            -25,
            30,
            45,
            75,
        ),  # (min_longitude, min_latitude, max_longitude, max_latitude)
    )
    ax.set_aspect(1.4)
    ax.set_title(get_meta(hue))